In [13]:
from pymoo.operators.sampling.lhs import LHS
from pymoo.core.problem import Problem
import numpy as np
import pandas as pd

In [9]:
class CustomProblem(Problem):
    def __init__(self):
        super().__init__(n_var=2, n_obj=2,xl=-10,xu=10)

    def _evaluate(self, x, out, *args, **kwargs):
        objective_values = []
        if(len(x)==1):
            objective_1 = x[0]**3 + x[1]**2
            objective_2 = x[1]**2 + 4*x[0]
            objective_values = [objective_1,objective_2]
        else:
            for  i in range(len(x)):
                value = x[i]
                objective_1 = value[0]**3 + value[1]**2
                objective_2 = value[1]**2 + 4*value[0]
                objective_values.append([objective_1,objective_2])
        out["F"] = objective_values

In [11]:
sampling_function = LHS()
problem = CustomProblem()
samples = sampling_function(problem,20).get('X')

In [16]:
dataset = pd.DataFrame(data=samples,columns=['x1','x2'])
out = {}
objectives = []
problem._evaluate(samples,out)
objectives = out['F']
objective_1 = [i[0] for i in objectives]
objective_2 = [i[1] for i in objectives]
dataset['Objective_1'] = objective_1
dataset['Objective_2'] = objective_2
dataset.head()

,x1,x2,Objective_1,Objective_2
0,2.947755,-3.129029,35.404640,21.581843
1,-4.274019,8.326991,-8.735765,52.242705
2,-5.027075,-1.737284,-124.023465,-17.090144
3,-6.751610,9.964879,-208.468120,72.292377
4,8.437050,-6.985087,649.372822,82.539638
